In [ ]:
from google.colab import files


files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"princegarg789","key":"bd4baf0871c3d0c6eeab2c7467279b80"}'}

In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d mohamedmustafa/real-life-violence-situations-dataset

Dataset URL: https://www.kaggle.com/datasets/mohamedmustafa/real-life-violence-situations-dataset
License(s): copyright-authors
100% 3.57G/3.58G [00:50<00:00, 116MB/s]
100% 3.58G/3.58G [00:50<00:00, 75.9MB/s]


In [ ]:
!unzip real-life-violence-situations-dataset

Archive:  real-life-violence-situations-dataset.zip
  inflating: Real Life Violence Dataset/NonViolence/NV_1.mp4  
  inflating: Real Life Violence Dataset/NonViolence/NV_10.mp4  
  inflating: Real Life Violence Dataset/NonViolence/NV_100.mp4  
  inflating: Real Life Violence Dataset/NonViolence/NV_1000.mp4  
  inflating: Real Life Violence Dataset/NonViolence/NV_101.mp4  
  inflating: Real Life Violence Dataset/NonViolence/NV_102.mp4  
  inflating: Real Life Violence Dataset/NonViolence/NV_103.mp4  
  inflating: Real Life Violence Dataset/NonViolence/NV_104.mp4  
  inflating: Real Life Violence Dataset/NonViolence/NV_105.mp4  
  inflating: Real Life Violence Dataset/NonViolence/NV_106.mp4  
  inflating: Real Life Violence Dataset/NonViolence/NV_107.mp4  
  inflating: Real Life Violence Dataset/NonViolence/NV_108.mp4  
  inflating: Real Life Violence Dataset/NonViolence/NV_109.mp4  
  inflating: Real Life Violence Dataset/NonViolence/NV_11.mp4  
  inflating: Real Life Violence Dataset/N

In [ ]:
import cv2
import os
import numpy as np
import tensorflow as tf
from glob import glob
import matplotlib.pyplot as plt
import shutil
import random
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Bidirectional, BatchNormalization, Dropout, Dense
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import IPython.display as ipd
from sklearn.metrics import classification_report
import subprocess
from tensorflow.keras.utils import to_categorical
from keras.applications import VGG16
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D


In [ ]:
# Function to check if a video meets the criteria
def meets_criteria(video_path):
    # Check if video contains at least 1 second of content
    cap = cv2.VideoCapture(video_path)
    success, _ = cap.read()
    cap.release()
    return success

In [ ]:
# Function to move video to appropriate folder based on criteria
def move_video(video_path, destination_dir):
    filename = os.path.basename(video_path)
    destination_path = os.path.join(destination_dir, filename)
    shutil.move(video_path, destination_path)


In [ ]:
def convert_mp4_to_avi(input_file, output_file):
    command = ['ffmpeg', '-i', input_file, output_file]
    subprocess.run(command)

In [ ]:
# Directory paths
NonViolenceVideos_Dir = "/content/Real Life Violence Dataset/NonViolence"
ViolenceVideos_Dir = "/content/Real Life Violence Dataset/Violence"
AllVideos_Dir = "/content/Real Life Violence Dataset/AllVideos"

In [ ]:
# Convert NonViolence videos
for video in os.listdir(NonViolenceVideos_Dir):
    video_path = os.path.join(NonViolenceVideos_Dir, video)
    if video.endswith('.mp4'):
        avi_path = os.path.splitext(video_path)[0] + '.avi'
        convert_mp4_to_avi(video_path, avi_path)
        os.remove(video_path)

# Convert Violence videos
for video in os.listdir(ViolenceVideos_Dir):
    video_path = os.path.join(ViolenceVideos_Dir, video)
    if video.endswith('.mp4'):
        avi_path = os.path.splitext(video_path)[0] + '.avi'
        convert_mp4_to_avi(video_path, avi_path)
        os.remove(video_path)

In [ ]:
for video in os.listdir(AllVideos_Dir):
    video_path = os.path.join(AllVideos_Dir, video)
    if meets_criteria(video_path):
        all_videos.append(video)

FileNotFoundError: [Errno 2] No such file or directory: '/content/Real Life Violence Dataset/AllVideos'

In [ ]:
# Move NonViolence videos
for file_name in os.listdir(NonViolenceVideos_Dir):
    source_path = os.path.join(NonViolenceVideos_Dir, file_name)
    destination_path = os.path.join(AllVideos_Dir, file_name)
    shutil.move(source_path, destination_path)

# Move Violence videos
for file_name in os.listdir(ViolenceVideos_Dir):
    source_path = os.path.join(ViolenceVideos_Dir, file_name)
    destination_path = os.path.join(AllVideos_Dir, file_name)
    shutil.move(source_path, destination_path)

In [ ]:
# Your existing code

# Frame size
img_size = 224

img_size_tuple = (img_size, img_size)

# Number of channels (RGB)
num_channels = 3

# Flat frame size
img_size_flat = img_size * img_size * num_channels

# Number of classes for classification (Violence-No Violence)
num_classes = 2

# Number of files to train
_num_files_train = 1

# Number of frames per video
_images_per_file = 20

# Number of frames per training set
_num_images_train = _num_files_train * _images_per_file

# Video extension
video_exts = ".avi"


In [ ]:
def get_frames(current_dir, file_name):
    images = []
    in_file = os.path.join(current_dir, file_name)
    vidcap = cv2.VideoCapture(in_file)
    success, image = vidcap.read()
    count = 0

    while count < _images_per_file and success:
        # Convert BGR to RGB
        rgb_img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        # Resize image
        resized_img = cv2.resize(rgb_img, dsize=img_size_tuple, interpolation=cv2.INTER_CUBIC)
        # Append to list
        images.append(resized_img)
        # Read next frame
        success, image = vidcap.read()
        count += 1

    # Convert list of images to numpy array
    result = np.array(images)
    # Normalize pixel values
    result = (result / 255.).astype(np.float16)

    return result

In [ ]:
import os
from random import shuffle

def label_video_names(in_dir):
    names = []
    labels = []

    if not os.path.exists(in_dir):
        print(f"Error: Directory '{in_dir}' does not exist.")
        return names, labels

    for file_name in os.listdir(in_dir):
        if file_name.startswith('V_'):
            labels.append([1, 0])  # Violence
            names.append(file_name)
        else:
            labels.append([0, 1])  # No violence
            names.append(file_name)

    if len(names) == 0:
        print(f"Error: No video files found in directory '{in_dir}'.")

    c = list(zip(names, labels))
    shuffle(c)
    names, labels = zip(*c)

    return names, labels


In [ ]:
# Define the directory where your videos are located
videos_dir = "/content/Real Life Violence Dataset/AllVideos"

# Get the names and labels of the whole videos
names, labels = label_video_names(videos_dir)

In [ ]:
labels[12]

In [ ]:
frames = get_frames(videos_dir, names[12])

In [ ]:
visible_frame = (frames*255).astype('uint8')

In [ ]:
plt.imshow(visible_frame[3])

#The VGG16 model contains a convolutional part and a fully-connected (or dense) part which is used for classification. If include_top=True then the whole VGG16 model is downloaded which is about 528 MB. If include_top=False then only the convolutional part of the VGG16 model is downloaded which is just 57 MB.


In [ ]:
image_model = VGG16(include_top=True, weights='imagenet')

In [ ]:
image_model.summary()

In [ ]:
input_shape = image_model.layers[0].output_shape[1:3]
input_shape

In [ ]:
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K

In [ ]:
# We will use the output of the layer prior to the final
# classification-layer which is named fc2. This is a fully-connected (or dense) layer.
transfer_layer = image_model.get_layer('fc2')

image_model_transfer = Model(inputs=image_model.input,
                             outputs=transfer_layer.output)

transfer_values_size = K.int_shape(transfer_layer.output)[1]


print("The input of the VGG16 net have dimensions:",K.int_shape(image_model.input)[1:3])

print("The output of the selecter layer of VGG16 net have dimensions: ", transfer_values_size)

In [ ]:
def get_transfer_values(current_dir, file_name):

    # Pre-allocate input-batch-array for images.
    shape = (_images_per_file,) + img_size_tuple + (3,)

    image_batch = np.zeros(shape=shape, dtype=np.float16)

    image_batch = get_frames(current_dir, file_name)

    # Pre-allocate output-array for transfer-values.
    # Note that we use 16-bit floating-points to save memory.
    shape = (_images_per_file, transfer_values_size)
    transfer_values = np.zeros(shape=shape, dtype=np.float16)

    transfer_values = \
            image_model_transfer.predict(image_batch)

    return transfer_values

In [ ]:
def proces_transfer(vid_names, in_dir, labels):

    count = 0

    tam = len(vid_names)

    # Pre-allocate input-batch-array for images.
    shape = (_images_per_file,) + img_size_touple + (3,)

    while count<tam:

        video_name = vid_names[count]

        image_batch = np.zeros(shape=shape, dtype=np.float16)

        image_batch = get_frames(in_dir, video_name)

         # Note that we use 16-bit floating-points to save memory.
        shape = (_images_per_file, transfer_values_size)
        transfer_values = np.zeros(shape=shape, dtype=np.float16)

        transfer_values = \
            image_model_transfer.predict(image_batch)

        labels1 = labels[count]

        aux = np.ones([20,2])

        labelss = labels1*aux

        yield transfer_values, labelss

        count+=1


In [ ]:
def make_files(n_files):

    gen = proces_transfer(names_training, videos_dir, labels_training)

    numer = 1

    # Read the first chunk to get the column dtypes
    chunk = next(gen)

    row_count = chunk[0].shape[0]
    row_count2 = chunk[1].shape[0]

    with h5py.File('prueba.h5', 'w') as f:

        # Initialize a resizable dataset to hold the output
        maxshape = (None,) + chunk[0].shape[1:]
        maxshape2 = (None,) + chunk[1].shape[1:]


        dset = f.create_dataset('data', shape=chunk[0].shape, maxshape=maxshape,
                                chunks=chunk[0].shape, dtype=chunk[0].dtype)

        dset2 = f.create_dataset('labels', shape=chunk[1].shape, maxshape=maxshape2,
                                 chunks=chunk[1].shape, dtype=chunk[1].dtype)

         # Write the first chunk of rows
        dset[:] = chunk[0]
        dset2[:] = chunk[1]

        for chunk in gen:

            if numer == n_files:

                break

            # Resize the dataset to accommodate the next chunk of rows
            dset.resize(row_count + chunk[0].shape[0], axis=0)
            dset2.resize(row_count2 + chunk[1].shape[0], axis=0)

            # Write the next chunk
            dset[row_count:] = chunk[0]
            dset2[row_count:] = chunk[1]

            # Increment the row count
            row_count += chunk[0].shape[0]
            row_count2 += chunk[1].shape[0]

            print_progress(numer, n_files)

            numer += 1

In [ ]:
import h5py
import sys

In [ ]:
def print_progress(count, max_count):
    # Percentage completion.
    pct_complete = count / max_count

    # Status-message. Note the \r which means the line should
    # overwrite itself.
    msg = "\r- Progress: {0:.1%}".format(pct_complete)

    # Print it.
    sys.stdout.write(msg)
    sys.stdout.flush()

In [ ]:
def make_files(n_files):
    gen = proces_transfer(names_training, videos_dir, labels_training)
    numer = 1

    # Read the first chunk to get the column dtypes
    chunk = next(gen)
    row_count = chunk[0].shape[0]
    row_count2 = chunk[1].shape[0]

    with h5py.File('prueba.h5', 'w') as f:
        # Initialize a resizable dataset to hold the output
        maxshape = (None,) + chunk[0].shape[1:]
        maxshape2 = (None,) + chunk[1].shape[1:]

        dset = f.create_dataset('data', shape=chunk[0].shape, maxshape=maxshape,
                                chunks=chunk[0].shape, dtype=chunk[0].dtype)

        dset2 = f.create_dataset('labels', shape=chunk[1].shape, maxshape=maxshape2,
                                 chunks=chunk[1].shape, dtype=chunk[1].dtype)

        # Write the first chunk of rows
        dset[:] = chunk[0]
        dset2[:] = chunk[1]

        for chunk in gen:
            if numer == n_files:
                break

            # Resize the datasets to accommodate the next chunk of rows
            dset.resize(row_count + chunk[0].shape[0], axis=0)
            dset2.resize(row_count2 + chunk[1].shape[0], axis=0)

            # Write the next chunk
            dset[row_count:] = chunk[0]
            dset2[row_count2:] = chunk[1]

            # Increment the row count
            row_count += chunk[0].shape[0]
            row_count2 += chunk[1].shape[0]

            print_progress(numer, n_files)
            numer += 1


In [ ]:
training_set = int(len(names)*0.8)
test_set = int(len(names)*0.2)

names_training = names[0:training_set]
names_test = names[training_set:]

labels_training = labels[0:training_set]
labels_test = labels[training_set:]

In [ ]:
make_files(training_set)

In [ ]:
make_files_test(test_set)

In [ ]:
def process_alldata_training():

    joint_transfer=[]
    frames_num=20
    count = 0

    with h5py.File('prueba.h5', 'r') as f:

        X_batch = f['data'][:]
        y_batch = f['labels'][:]

    for i in range(int(len(X_batch)/frames_num)):
        inc = count+frames_num
        joint_transfer.append([X_batch[count:inc],y_batch[count]])
        count =inc

    data =[]
    target=[]

    for i in joint_transfer:
        data.append(i[0])
        target.append(np.array(i[1]))

    return data, target

In [ ]:
def process_alldata_test():

    joint_transfer=[]
    frames_num=20
    count = 0

    with h5py.File('pruebavalidation.h5', 'r') as f:

        X_batch = f['data'][:]
        y_batch = f['labels'][:]

    for i in range(int(len(X_batch)/frames_num)):
        inc = count+frames_num
        joint_transfer.append([X_batch[count:inc],y_batch[count]])
        count =inc

    data =[]
    target=[]

    for i in joint_transfer:
        data.append(i[0])
        target.append(np.array(i[1]))

    return data, target

In [ ]:
data, target = process_alldata_training()
data_test, target_test = process_alldata_test()

In [ ]:
from keras.models import Sequential
from keras.layers import Activation

In [ ]:
chunk_size = 4096
n_chunks = 20
rnn_size = 512

model = Sequential()
model.add(LSTM(rnn_size, input_shape=(n_chunks, chunk_size)))
model.add(Dense(1024))
model.add(Activation('relu'))
model.add(Dense(50))
model.add(Activation('sigmoid'))
model.add(Dense(2))
model.add(Activation('softmax'))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
epoch = 200
batchS = 500
history = model.fit(np.array(data[0:750]), np.array(target[0:750]), epochs=epoch,
                    validation_data=(np.array(data[750:]), np.array(target[750:])),
                    batch_size=batchS, verbose=2)

In [ ]:
result = model.evaluate(np.array(data_test), np.array(target_test))

In [ ]:
for name, value in zip(model.metrics_names, result):
    print(name, value)

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.savefig('destination_path.eps', format='eps', dpi=1000)
plt.show()

In [ ]:
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.savefig('destination_path1.eps', format='eps', dpi=1000)
plt.show()